# Subclassing API

In [1]:
import tensorflow as tf;import tensorflow.keras as keras

C:\Users\SONY\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SONY\miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\SONY\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class wideandeepmodel(keras.models.Model):

    def __init__(self,units=30,activacion='relu',**kwargs):
        super().__init__(**kwargs)

        self.hidden1= keras.layers.Dense(units=units,activation=activacion)
        self.hidden2=keras.layers.Dense(units=units,activation=activacion)
        self.main_ouput=keras.layers.Dense(1)
        self.aux_ouput=keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):
        input_a,input_b=inputs
        h1=self.hidden1(input_b)
        h2=self.hidden2(h1)
        conca=keras.layers.concatenate([input_a,h2])
        main_output=self.main_ouput(conca)
        aux_output=self.aux_ouput(h2)

        return main_output,aux_output

In [2]:
from sklearn.datasets import fetch_california_housing;from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X,y= fetch_california_housing(return_X_y=True)

X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42)

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
modelo=wideandeepmodel()


In [6]:
modelo.compile(loss='mean_squared_error',optimizer=keras.optimizers.SGD(1e-3))


In [7]:
X_train_A, X_train_B = X_train_scaled[:, :5], X_train_scaled[:, 2:]
X_test_A, X_test_B = X_test_scaled[:, :5], X_test_scaled[:, 2:]

In [8]:
hist=modelo.fit(x=[X_train_A,X_train_B],y=[y_train,y_train],validation_split=0.2,epochs=20)


Epoch 1/20
387/387 [==============================] - 1s 2ms/step - loss: 4.5433 - output_1_loss: 2.0259 - output_2_loss: 2.5175 - val_loss: 2.0444 - val_output_1_loss: 0.7885 - val_output_2_loss: 1.2559
Epoch 2/20
387/387 [==============================] - 0s 1ms/step - loss: 1.7977 - output_1_loss: 0.6840 - output_2_loss: 1.1137 - val_loss: 1.7258 - val_output_1_loss: 0.6700 - val_output_2_loss: 1.0558
Epoch 3/20
387/387 [==============================] - 0s 1ms/step - loss: 1.5808 - output_1_loss: 0.6172 - output_2_loss: 0.9636 - val_loss: 1.5663 - val_output_1_loss: 0.6317 - val_output_2_loss: 0.9346
Epoch 4/20
387/387 [==============================] - 0s 1ms/step - loss: 1.4561 - output_1_loss: 0.5861 - output_2_loss: 0.8701 - val_loss: 1.4614 - val_output_1_loss: 0.6046 - val_output_2_loss: 0.8568
Epoch 5/20
387/387 [==============================] - 0s 1ms/step - loss: 1.3697 - output_1_loss: 0.5607 - output_2_loss: 0.8090 - val_loss: 1.3863 - val_output_1_loss: 0.5823 - val_ou

In [9]:
y_pred1,y_pred2=modelo.predict([X_test_A,X_test_B])

In [10]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred1)

0.6626286694073888

In [11]:
r2_score(y_test,y_pred2)

0.5468959631840093

# Callbacks

In [12]:
modelo=keras.models.Sequential([keras.layers.Dense(30,activation='relu',
                                                   input_shape=[X_train.shape[1]]),
                                keras.layers.Dense(1)])

modelo.compile(loss='mean_squared_error',optimizer=keras.optimizers.SGD(1e-3))

early_stopping= keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

histo= modelo.fit(x=X_train_scaled,y=y_train,validation_split=0.2,epochs=100,callbacks=[early_stopping])

Epoch 1/100
387/387 [==============================] - 0s 1ms/step - loss: 2.0501 - val_loss: 0.8718
Epoch 2/100
387/387 [==============================] - 0s 939us/step - loss: 0.7599 - val_loss: 0.7209
Epoch 3/100
387/387 [==============================] - 0s 929us/step - loss: 0.6775 - val_loss: 0.6802
Epoch 4/100
387/387 [==============================] - 0s 934us/step - loss: 0.6398 - val_loss: 0.6493
Epoch 5/100
387/387 [==============================] - 0s 920us/step - loss: 0.6105 - val_loss: 0.6236
Epoch 6/100
387/387 [==============================] - 0s 1ms/step - loss: 0.5867 - val_loss: 0.6021
Epoch 7/100
387/387 [==============================] - 0s 901us/step - loss: 0.5669 - val_loss: 0.5842
Epoch 8/100
387/387 [==============================] - 0s 925us/step - loss: 0.5502 - val_loss: 0.5701
Epoch 9/100
387/387 [==============================] - 0s 909us/step - loss: 0.5362 - val_loss: 0.5576
Epoch 10/100
387/387 [==============================] - 0s 935us/step - loss:

In [13]:
r2_score(y_test,modelo.predict(X_test_scaled))

0.7010211105659221

# Visualization Using TensorBoard

In [4]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [5]:
run_logdir = get_run_logdir()

In [6]:
modelo=keras.models.Sequential([keras.layers.Dense(30,activation='relu',
                                                   input_shape=[X_train.shape[1]]),
                                keras.layers.Dense(1)])

modelo.compile(loss='mean_squared_error',optimizer=keras.optimizers.SGD(1e-3))

tensorboard= keras.callbacks.TensorBoard(run_logdir)

histo= modelo.fit(x=X_train_scaled,y=y_train,validation_split=0.2,epochs=100,callbacks=[tensorboard])

Epoch 1/100
  1/387 [..............................] - ETA: 0s - loss: 6.1454WARNING:tensorflow:From C:\Users\SONY\miniconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
387/387 [==============================] - 1s 3ms/step - loss: 2.2206 - val_loss: 1.1596
Epoch 2/100
387/387 [==============================] - 0s 1ms/step - loss: 0.9415 - val_loss: 0.8449
Epoch 3/100
387/387 [==============================] - 0s 1ms/step - loss: 0.7743 - val_loss: 0.7677
Epoch 4/100
387/387 [==============================] - 0s 955us/step - loss: 0.7198 - val_loss: 0.7265
Epoch 5/100
387/387 [==============================] - 0s 974us/step - loss: 0.6843 - val_loss: 0.6932
Epoch 6/100
387/387 [==============================] - 0s 998us/step - loss: 0.6551 - val_loss: 0.6658
Epoch 7/100
387/387 [====================

387/387 [==============================] - 0s 995us/step - loss: 0.4086 - val_loss: 0.4452
Epoch 74/100
387/387 [==============================] - 0s 954us/step - loss: 0.4083 - val_loss: 0.4444
Epoch 75/100
387/387 [==============================] - 0s 992us/step - loss: 0.4074 - val_loss: 0.4436
Epoch 76/100
387/387 [==============================] - 0s 945us/step - loss: 0.4068 - val_loss: 0.4431
Epoch 77/100
387/387 [==============================] - 0s 938us/step - loss: 0.4060 - val_loss: 0.4430
Epoch 78/100
387/387 [==============================] - 0s 954us/step - loss: 0.4055 - val_loss: 0.4419
Epoch 79/100
387/387 [==============================] - 0s 942us/step - loss: 0.4047 - val_loss: 0.4414
Epoch 80/100
387/387 [==============================] - 0s 998us/step - loss: 0.4041 - val_loss: 0.4410
Epoch 81/100
387/387 [==============================] - 0s 921us/step - loss: 0.4034 - val_loss: 0.4403
Epoch 82/100
387/387 [==============================] - 0s 955us/step - loss:

In [10]:
import numpy as np
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)